# Getting started

First, we enable the cluster to scale up. Note that if you run an auto-scaling cluster,
Google will suspend your nodes. Make sure to have the experiment prepared before running the commands.

The following is assumed ready:
* GKE/Kubernetes cluster (see also `terraform/terraform_notebook.ipynb`)
    * 2 nodes pools (default for system & dependencies, experiment pool)
* Docker image (including dataset, to speed-up starting experiments).
    * Within a bash shell
        * Make sure to have the `requirements-cpu.txt` installed (or `requirements-gpu.txt (in a virtual venv/conda environment). You can run `pip3 install -r requirements-cpu.txt`
    * First run the extractor (locally) `python3 -m fltk extractor configs/example_cloud_experiment.json`
        *  This downloads datasets to be included in the docker image.
    * Build the container `DOCKER_BUILDKIT=1 docker build --platform linux/amd64 . --tag gcr.io/$PROJECT_ID/fltk`
    * Push to your gcr.io repository `docker push gcr.io/$PROJECT_ID/fltk`


With that setup, first set some variables used throughout the experiment.


In [23]:
##################
### CHANGE ME! ###
##################
PROJECT_ID="eric-cs4215-fltk"
CLUSTER_NAME="fltk-testbed-cluster"
DEFAULT_POOL="default-node-pool"
EXPERIMENT_POOL="medium-fltk-pool-1"
REGION="us-central1-c"
alias gcloud=/home/yifan/.local/tools/google-cloud-sdk/bin/gcloud

# In case we do not yet have the credentials/kubeconfig
gcloud container clusters get-credentials $CLUSTER_NAME --region $REGION --project $PROJECT_ID

Fetching cluster endpoint and auth data.
kubeconfig entry generated for fltk-testbed-cluster.


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



Scale the default-node-pool up.

In [43]:
# These commands might take a while to complete.
gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
     --num-nodes 1 --region $REGION --quiet

Resizing fltk-testbed-cluster...done.                                          
Updated [https://container.googleapis.com/v1/projects/eric-cs4215-fltk/zones/us-central1-c/clusters/fltk-testbed-cluster].


## Preparation
In case you have already tested something or ran another experiment, we have to remove the deployment of the Orchestrator. This will not delete any experiment data, as this persists on one of the ReadWriteMany PVCs.


Currently, the Orchestrator is deployed using a `Deployment` definition, a future version will replace this with a `Deployment` definition, to make this step unnecessary. For experiments this means the following:

1. A single deployment can exist at a single time in a single namespace. This includes 'completed' experiments.
2. For running batches of experiments, a BatchOrchestrator is provided.


ℹ️ This will not remove any data, but if your orchestrator is still/already running experiments, this will stop the deployment. Running training jobs will not be stopped, for this you can use `kubectl`. ConfigMaps created by the Orchestrator (to provide experiment configurations), will not be removed. See the commented code in the cell below.

In [2]:
# If you want to delete all pytorch trainjobs, uncomment the command below.
# kubectl delete pytorchjobs.kubeflow.org --all --namespace test

# If you want to delete all existing configuration map objects in a namespace, run teh command below
# kubectl delete configmaps --all --namespace test

helm uninstall -n test flearner

W1007 12:01:09.427171    8278 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: flearner: release: not found
W1007 12:01:10.605377    8306 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Install extractor

Deploy the TensorBoard service and persistent volumes, required for deployment of the orchestrator's chart.

In [3]:
helm upgrade --install -n test extractor ../charts/extractor -f ../charts/fltk-values.yaml \
    --set provider.projectName=$PROJECT_ID

W1007 12:01:14.327713    8345 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Release "extractor" has been upgraded. Happy Helming!
NAME: extractor
LAST DEPLOYED: Fri Oct  7 12:01:16 2022
NAMESPACE: test
STATUS: deployed
REVISION: 5
TEST SUITE: None
NOTES:
Get the FLTK extractors Tensorboard URL by running:

export POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
echo http://localhost:6006/
kubectl -n test port-forward $POD_NAME 6006:6006


## Define experiment configuration files

Deployment of experiments is currently done through a Helm Deployment. A future release (™️) will rework this to a Job definition, as this allows to re-use the template more easily.


> The `EXPERIMENT_FILE` will contain the description of the experiments
> The `CLUSTER_CONFIG` will contain shared configurations for logging, Orchestrator configuration and replication information.

In [24]:
EXPERIMENT_FILE="../configs/distributed_tasks/example_arrival_config.json"
CLUSTER_CONFIG="../configs/example_cloud_experiment.json"

## Setup experiment variables
Next, we will deploy the experiments.


We provide a configuration file, `charts/fltk-values.yaml`, in here change the values under the `provider` block. Change `projectName` to your Google Cloud Project ID.

```yaml
provider:
    domain: gcr.io
    projectName: CHANGE_ME!
    imageName: fltk:latest
```

We use the `--set-file` flag for `helm`, as currently, Helm does not support using files outside of the chart root directory (in this case `charts/orchestrator`). Using `--set-file` we can dynamically provide these files. See also issue [here](https://github.com/helm/helm/issues/3276)


In [25]:
echo $EXPERIMENT_FILE
echo $CLUSTER_CONFIG
echo $PROJECT_ID

../configs/distributed_tasks/example_arrival_config.json
../configs/example_cloud_experiment.json
eric-cs4215-fltk


In [26]:
helm uninstall -n test experiment-orchestrator
helm install -n test experiment-orchestrator ../charts/orchestrator -f ../charts/fltk-values.yaml \
    --set-file orchestrator.experiment=$EXPERIMENT_FILE,orchestrator.configuration=$CLUSTER_CONFIG \
    --set provider.projectName=$PROJECT_ID

W1007 15:49:51.506499   26525 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
Error: uninstall: Release not loaded: experiment-orchestrator: release: not found
W1007 15:49:52.575535   26555 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME: experiment-orchestrator
LAST DEPLOYED: Fri Oct  7 15:49:54 2022
NAMESPACE: test
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
You successfully launched an experiment configuration on your cluster in test.

N.B. Make sure to collect all data after completing your experiment!
N.B. Re-installing the orchestrator WILL RESULT IN DELETION OF ALL TRAINJOBS and PODS!


In [20]:
kubectl get all -n test

W1007 15:32:21.570449   25537 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME                                                         READY   STATUS    RESTARTS   AGE
pod/fl-extractor-c5bcd78bd-2nhvw                             1/1     Running   0          47h
pod/fl-server                                                1/1     Running   0          3h30m
pod/nfs-server-nfs-server-provisioner-0                      1/1     Running   0          47h
pod/trainjob-13a09ce3-5806-4691-a9e6-a2d968e63475-master-0   0/1     Pending   0          3h22m
pod/trainjob-13a09ce3-5806-4691-a9e6-a2d968e63475-worker-0   0/1     Pending   0          3h22m
pod/trainjob-304f1413-b894-4dc7-a3d1-70d9d997309e-master-0   0/1     Pending   0          3h22m
pod/trainjob-304f1413-b894-4dc7-a3d1-70d9d997309e-worker-0   0/1     Pending   0   

In [8]:
helm ls -n=test

W1007 12:01:40.470686    8441 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
NAME                   	NAMESPACE	REVISION	UPDATED                                 	STATUS  	CHART                         	APP VERSION
experiment-orchestrator	test     	1       	2022-10-07 12:01:28.660963903 +0200 CEST	deployed	../docker-compose-gcloud-0.3.0	1.17.0     
extractor              	test     	5       	2022-10-07 12:01:16.723639247 +0200 CEST	deployed	fltk-extractor-0.1.0          	1.17.0     
nfs-server             	test     	1       	2022-10-05 16:05:30.825424619 +0200 CEST	deployed	nfs-server-provisioner-1.1.3  	2.3.0      


In [21]:
# To get logs from the orchestrator
kubectl logs -n test fl-server

W1007 15:35:58.784761   25873 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-07-2022 10:01:46 root         INFO     Loading file config/configuration.fltk.json
No argument path is provided.
10-07-2022 10:01:46 root         INFO     Starting in cluster mode.
10-07-2022 10:01:46 root         INFO     Starting with experiment replication: 0 with seed: 42
10-07-2022 10:01:46 root         INFO     Starting as Orchestrator
10-07-2022 10:01:46 root         INFO     Starting Orchestrator, initializing resources....
10-07-2022 10:01:46 root         INFO     Loading in cluster configuration file
10-07-2022 10:01:46 root         INFO     Pointing configuration to in cluster configuration.
10-07-2022 10:01:46 root         INFO     Starting cluster manager
10-07-2022 10:01:46 ClusterManager INFO     Spinning up cluster manag

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [17]:
# To get logs from learners (example)
kubectl logs -n test trainjob-6953e293-ac11-4ec5-b33b-d843f46e3f49-master-0

# To get logs from learners (federated learning)
kubectl logs -n test trainjob-6953e293-ac11-4ec5-b33b-d843f46e3f49-worker-0

W1007 12:13:23.818011   10343 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
10-07-2022 10:06:43 root         INFO     Loading file config/configuration.fltk.json
10-07-2022 10:06:43 root         INFO     Starting in client mode
10-07-2022 10:06:43 root         INFO     Starting with host=trainjob-6953e293-ac11-4ec5-b33b-d843f46e3f49-master-0 and port=23456
10-07-2022 10:06:43 root         INFO     Initializing backend for training process: gloo
10-07-2022 10:06:44 torch.distributed.distributed_c10d INFO     Added key: store_based_barrier_key:1 to store for rank: 0
10-07-2022 10:06:44 torch.distributed.distributed_c10d INFO     Rank 0: Completed store-based barrier for key:store_based_barrier_key:1 with 2 nodes.
10-07-2022 10:06:44 root         INFO     Starting Creating client with 0
10-07-2022 10:06:44 Client-0-6

## Copy experiment results from the extractor

Extractor holds the experiment results in the format that can be processedby TensorBoard.
In order to download it to the local machine, execute:

In [18]:
EXTRACTOR_POD_NAME=$(kubectl get pods -n test -l "app.kubernetes.io/name=fltk.extractor" -o jsonpath="{.items[0].metadata.name}")
kubectl cp -n test $EXTRACTOR_POD_NAME:logging ./logging

W1007 12:13:30.001931   10381 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
W1007 12:13:30.594607   10402 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke


In [51]:
ls ./logging

In [7]:
kubectl exec -ti -n test pod/$EXTRACTOR_POD_NAME -- curl localhost:8080

W1006 13:47:56.951027   72086 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
error: Internal error occurred: error executing command in container: failed to exec in container: failed to start exec "592c969f7b577626eb074104fb34ace57da31d8b018ccf8cfb958f4be1226a05": OCI runtime exec failed: exec failed: container_linux.go:380: starting container process caused: exec: "curl": executable file not found in $PATH: unknown


: 1

# Cleanup

## Removing orchestrator

In [27]:
helm uninstall -n test experiment-orchestrator

W1007 15:52:34.572307   27016 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "experiment-orchestrator" uninstalled


## Removing extractor

IMPORTANT: Removing extractor chart will result in deleting the already collected experiment results, stored in the NFS!

In [28]:
helm uninstall extractor -n test

W1007 15:52:37.275032   27040 gcp.go:120] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.25+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
release "extractor" uninstalled


## Wrapping up

To scale down the cluster nodepools, run the cell below. This will scale the node pools down and remove all the experiments deployed (on the cluster).

1. Experiments cannot be restarted.
2. Experiment logs will not persist deletion.


In [29]:
# This will remove all information and logs as well.
kubectl delete pytorchjobs.kubeflow.org --all-namespaces --all

gcloud container clusters resize $CLUSTER_NAME --node-pool $DEFAULT_POOL \
    --num-nodes 0 --region $REGION --quiet

gcloud container clusters resize $CLUSTER_NAME --node-pool $EXPERIMENT_POOL \
    --num-nodes 0 --region $REGION --quiet

W1007 15:52:40.921053   27060 gcp.go:119] WARNING: the gcp auth plugin is deprecated in v1.22+, unavailable in v1.26+; use gcloud instead.
To learn more, consult https://cloud.google.com/blog/products/containers-kubernetes/kubectl-auth-changes-in-gke
pytorchjob.kubeflow.org "trainjob-13a09ce3-5806-4691-a9e6-a2d968e63475" deleted
pytorchjob.kubeflow.org "trainjob-304f1413-b894-4dc7-a3d1-70d9d997309e" deleted
pytorchjob.kubeflow.org "trainjob-4403c888-2f87-4281-8ff8-1eff75b9dbfb" deleted
pytorchjob.kubeflow.org "trainjob-6953e293-ac11-4ec5-b33b-d843f46e3f49" deleted
pytorchjob.kubeflow.org "trainjob-6a4f7ff2-1c49-4a0f-9e24-ec337c2c25e9" deleted
pytorchjob.kubeflow.org "trainjob-840e2c61-6dd9-467d-b83f-0cd88624ef58" deleted
pytorchjob.kubeflow.org "trainjob-a36821b5-9568-4f93-affa-d029e4d36de5" deleted
pytorchjob.kubeflow.org "trainjob-d42c1149-8061-4d39-b115-95db44509454" deleted
pytorchjob.kubeflow.org "trainjob-e6b4d6f0-d40a-4053-aec8-9da2ee1664dc" deleted
Resizing fltk-testbed-cluster

: 1